# Rewriting Symbolic Expressions in Bartiq

Double factorization (DF) is an important subroutine in quantum computations of chemistry. This notebook is a deep dive into its guts and implementation from the quantum circuit perspective – it does _not_ go into any detail regarding the actual factorization protocol itself. It is intended to stand as a one stop shop for understanding the circuit implementatio



This is based mainly off of these two papers:

- [Quantum computing enhanced computational catalysis](https://arxiv.org/abs/2007.14460): This is the original paper with the implementation, and is **fantastic** for getting an understanding of all the subroutines and circuits involved with DF. The overall circuit is, however, suboptimal (and slightly wrong). **HEREAFTER THIS WILL BE REFERRED TO AS "THE MICROSOFT PAPER"**.
- [Even more efficient quantum computations of chemistry through tensor hypercontraction](https://arxiv.org/abs/2011.03494): This is the paper the circuits in Workbench are based off of, but its _much_ harder to reason through what goes where. **HEREAFTER THIS WILL BE REFERRED TO AS "THE GOOGLE PAPER"**.

## The goal of the subroutine
DF is nothing special – it's a block encoding of a Hamiltonian,

$$
U = \begin{bmatrix} H & * \\
* & *
\end{bmatrix} \ ,
$$

which may be used as a subroutine in other algorithms – most notably, it can be used as part of a qubitization iterate in the input to QPE to obtain eigenphases of $H$. The Hamiltonian we want to block encode is the electronic structure Hamiltonian used in quantum chemistry, expressed in a provocative form.

### Constructing the Hamiltonian
In this section, we provide a brief overview of the construction of the double factorized Hamiltonian – for the full details of the derivation, see the Microsoft paper (note that we use Microsoft's notation as it's clearer what's going on, but we use Google's version of the actual circuit, since it's more efficient - additionally we neglect any analysis of terms proportional to the identity, since they just result in an energy shift that can be applied in a classical post-processing step).

The canonical representation of the electronic structure Hamiltonian in second quantized form is

$$
H = \underbrace{\sum_{i, j = 1}^{N_{\text{spatial}}}\sum_{\sigma\in\{\uparrow, \downarrow\}} h_{ij} a^\dagger_{i, \sigma}a_{j, \sigma}}_{\text{One body}} + \underbrace{\sum_{i, j, k, l = 1}^{N_{\text{spatial}}}\sum_{\sigma,\rho\in\{\uparrow, \downarrow\}} h_{ijkl} a^\dagger_{i, \sigma}a^\dagger_{k, \rho}a_{l, \rho}a_{j, \sigma}}_{\text{Two body}} \ ,
$$

where $N_{\text{spatial}}$ is the number of spatial orbitals, $a_k, a_k^\dagger$ are the fermionic creation/annihilation operators acting on the $k$th mode and where the one and two electron integrals are defined as

\begin{align}
h_{ij} &= \int \psi_i^*(x_1)\left(-\frac{\nabla^2}{2}-\sum_m\frac{Z_m}{|x_1-r_m|}\right)\psi_j(x_1)d^3x_1 \\
h_{ijkl} &= \int \psi_i^*(x_1)\psi_j(x_1)\left(\frac{1}{|x_1 - x_2|}\right)\psi^*_k(x_2)\psi_l(x_2)d^3x_1d^3x_2 \ ,
\end{align}

where the one electron integral describes the interaction between electrons at position $x_1$ (represented using basis functions $\{\psi_k\}$) and nuclei with charges $Z_m$ and positions $r_m$ and the two electron integral described the interaction between electrons at positions $x_1$ and $x_2$.

The central insight of DF (and other factorization methods such as tensor hypercontraction) is that we can represent the two electron integrals much more efficiently by analyzing them as a single object – a 4-index tensor and performing a series of decompositions on it.

### Single factorization
The first thing we can do is factorize the 4-index tensor into a contraction of 2-index tensors. Explicitly, we do

$$
h_{ijkl} = \sum_{k=1}^{R}L^{(r)}_{ij}\left(L^{(r)}\right)^T_{kl}
$$

which we can achieve with either a Cholesky decomposition (motivating the common notation for quantum chemistry Hamiltonians that are only decomposed up to this point, $H_{CD}$) or a singular value decomposition. As written, if we set $R=N^2$, we have an exact equality, and we have gained relatively little (except, arguably, a more approachable expression to implement on a quantum computer). However, we can also truncate the decomposition at some $R < N^2$, making this the first axis over which we can gain some efficiency over the most naive block encoding approaches.

### Double factorization
The key insight of the Microsoft paper was to take this decomposition a step further, decomposing each of the $L^{(k)}_{ij}$ matrices into a sum of their eigenvectors:

$$
L^{(r)} = \sum_{k=1}^{M^{(r)}} \lambda_k^{L^{(r)}} \vec{R^{(r)}_k} \cdot \vec{R^{(r)}_k}^T
$$

where we can once again truncate at $M^{(r)} < N^2$ to obtain a block encoding with reduced costs. Though this is inherently advantageous, the Microsoft paper also introduced a technique for implementing the Coulomb operator in this form very efficiently through the use of basis rotated number operators in the Majorana basis. We will go over the details of this decomposition, as well as all the explicit circuits, in the remainder of this notebook. We will skip over a fair number of the techincal details in the derivation though, so interested readers are encouraged to read the references given above.

The main focus of our analysis will be on the double factorized chemistry Hamiltonian (ignoring terms proportional to the identity):

$$
H_{DF} = \underbrace{\frac{i}{2}\sum_{k=1}^{M^{(-1)}}\lambda^{L^{(-1)}}_k\sum_{\sigma\in\{\uparrow, \downarrow\}} \gamma_{\vec{R}^{(-1)}_k, \sigma, 0} \gamma_{\vec{R}^{(-1)}_k, \sigma, 1}}_{\text{ONE}_{L^{(-1)}}} + \frac{1}{4}\sum_{r=1}^R \vert\vert L^{(r)}\vert\vert^2_{SC}T_2\left[ \frac{1}{\vert\vert L^{(r)}\vert\vert_{SC} } \underbrace{\frac{i}{2} \sum_{k=1}^{M^{(r)}}\lambda^{L^{(r)}}_k\sum_{\sigma\in\{\uparrow, \downarrow\}} \gamma_{\vec{R}^{(r)}_k, \sigma, 0} \gamma_{\vec{R}^{(r)}_k, \sigma, 1}}_{\text{ONE}_{L^{(r)}}}\right]
$$

(For those following along with the paper, this is Eq. (54) in the Microsoft paper, expanded explicitly using Eq. (52)).

The symbols used here are:

- $M^{(k)}$: The rank of the $k$th single factorized matrix $L^{(k)}$
- $\lambda_k^{L^{(r)}}$: The $k$th eigenvalue of the $r$th factorized matrix such that $L^{(r)} = \sum_{k=1}^{M^{(r)}} \lambda_k^{L^{(r)}} \vec{R^{(r)}_k} \cdot \vec{R^{(r)}_k}^T$
- $\sigma$: Label for spin indices
- $N_{\text{spatial}}$: The number of spatial orbitals
- $R$: The rank of the single factorization (that is, the number of matrices $L^{(r)}$ that we have)
- $\gamma_{\vec{R}^{(r)}_k, \sigma, k}$: Basis-rotated Majorana representation of the fermionic creation and annihilation operators, $\gamma_{\vec{R}^{(r)}_k, \sigma, k}=\sum_{i=1}^{N_{\text{spatial}}}\left(\vec{R}^{(r)}_k\right)_i\gamma_{i, \sigma, k}$
- $\gamma_{j, \sigma, k}$: Majorana representation of the fermionic creation and annihilation operators, $\gamma_{j, \sigma, 0} := a_{j, \sigma} + a^\dagger_{j, \sigma}$, $\gamma_{j, \sigma, 1} := -i(a_{j, \sigma} + a^\dagger_{j, \sigma})$
- $T_2$: Second order Chebyshev polynomial (which can be implemented via a product of two qubitization iterates – see [Lin Lin's notes](https://arxiv.org/abs/2201.08309))
- $\vert\vert \cdot \vert\vert_{SC}$: Schatten norm, $\vert\vert h\vert\vert_{SC} := \sum_{k\in N} |\lambda_k|$ for all Hermitian $h\in\mathbb{C}^{N\times N}$, with $h = \sum_k \lambda_k |\lambda_k\rangle\langle\lambda_k|$ the eigendecomposition of $h$.

Our goal is to block encode this Hamiltonian.

## Wait, what _is_ that?
$H_{DF}$ looks like a beast (and, in fairness, it is), but it's really just the standard quantum chemistry Hamiltonian in second quantized form with a bunch of decompositions applied. Although it looks unpleasant, it's actually in quite a nice form for implementing on a quantum computer: it consists of two terms, a one-electron term and a two-electron term (noting that the argument inside the Chebyshev polynomial $T_2$ is essentially the same as the one-electron term, meaning that the two-electron term can be implemented using mostly the same machinery as the one-electron term – this is a key insight from the Google paper, and we will be exploring and making use of this fact shortly|):

$$
H_{DF} = \text{ONE}_{L^{(-1)}} + \sum_{r=1}^{R}T_2\left[\frac{\text{ONE}_{L^{(r)}}}{\vert\vert L^{(r)}\vert\vert_{SC}}\right]
$$

where

$$
\text{ONE}_{L^{(r)}} = \frac{i}{2} \sum_{k=1}^{M^{(r)}}\lambda^{L^{(r)}}_k\sum_{\sigma\in\{\uparrow, \downarrow\}} \gamma_{\vec{R}^{(r)}_k, \sigma, 0} \gamma_{\vec{R}^{(r)}_k, \sigma, 1}
$$


We have all the machinery to implement this (don't worry if you don't get all the details – we will dive into each part separately):
- The product of Majorana fermion raising and lowering operators can be implemented by performing a basis rotation using Givens rotations and applying a standard number operator under Jordan-Wigner (otherwise known as a $Z$ operator).
- Given that we can implement this operator (referred to hereafter as a basis-rotated number operator), we can implement a block encoding of $\text{ONE}_{L^{(r)}}$ using our standard $\text{PREP}-\text{SELECT}-\text{PREP}^\dagger$ framework by loading the eigenvalues $\lambda_k^{L^{(r)}}$ with some $\text{PREP}$ routine and treating the basis-rotated number operator as our $\text{SELECT}$, which essentially amounts to controlling the Givens rotations.
- Given that we can block encode $\text{ONE}_{L^{(r)}}$, one can obtain $T_2\left[\text{ONE}_{L^{(r)}}\right]$ by converting the block encoding to a qubitization iterate by multiplying by a $Z_{\Pi}$ operator (that is, an operator which does nothing in the $|0^{\otimes n}\rangle$ sector of the block encoding and applies a phase of $-1$ to the $|\perp\rangle$ sector) and applying the resulting operator twice.
- Given that we can block encode $T_2\left[\text{ONE}_{L^{(r)}}\right]$, we can block encode the two body term by using another $\text{PREP}-\text{SELECT}-\text{PREP}^\dagger$ construction with the $T_2\left[\text{ONE}_{L^{(r)}}\right]$ over different values of $r$ as $\text{SELECT}$ and with $\text{PREP}$ preparing the squares of the factorized matrix norms $\vert\vert L^{(r)}\vert\vert^2$, giving our 2 body terms.
- Given that we can block encode both our one and two body terms, we can block encode $H_{DF}$ by LCUing them together – **HOWEVER** there is a trick we can use that the Google folks noticed. Since the two body term is effectively the one body term twice, with a reflection in between, we can fold the one body term into the two body block encoding, adding one term to the outer $\text{PREP}-\text{SELECT}-\text{PREP}^\dagger$ (the zeroth term) and controlling the reflection and the second block encoding on the $\text{PREP}$ register being not equal to zero.

In the rest of this notebook, we'll go through the circuit, what data is passed where and how the Workbench implementation in QApps works.

# Set up definitions of symbolic parameters
num_spatial = Parameter("N_spatial", "The number of spatial orbitals.")
max_rank = Parameter(
    "M_r",
    "The maximum rank (i.e. number of eigenvectors) of the factorized matrices from the first (Cholesky or \
        singular value) decomposition of the Coulomb tensor (Xi^(l) in the Google paper)."
)
num_ranks = Parameter(
    "R",
    "The number of factorized matrices from the first (Cholesky or singular value) decomposition of the \
        Coulomb tensor (L in the Google paper)"
)
alias_sampling_bits_of_precision = Parameter(
    "b_as",
    "The number of bits of precision to represent the alt and keep values in alias sampling"
)
multiplexed_alias_sampling_bits_of_precision = Parameter(
    "b_mas",
    "The number of bits of precision to represent the alt and keep values in multiplexed alias sampling"
)
givens_rotation_bits_of_precision = Parameter(
    "b_givens",
    "The number of bits of precision to represent each Givens rotation in the basis rotated number operator. The \
        total number of bits of precision in the QROM that loads these rotations is (N_spatial - 1) * b_givens, \
            since all of the Givens rotations are loaded in parallel."
)

In [4]:
from bartiq import Routine, sympy_backend
from bartiq.compilation import compile_routine, CompilationFlags
from pathlib import Path
import yaml

try:
    root_dir = next(p for p in (Path.cwd().resolve(), *Path.cwd().resolve().parents) if p.parts[-1] == "bartiq")
    with open(root_dir / "tests/compilation/data/df_qref.yaml") as f:
        rout = Routine.from_qref(yaml.safe_load(f), sympy_backend)
except StopIteration as exc:
    raise ValueError(
        "Can't find the `bartiq` root directory! Make sure this notebook is somewhere in the `bartiq` repo."
    ) from exc

compiled_routine = compile_routine(rout, compilation_flags=CompilationFlags.EXPAND_RESOURCES).routine

In [5]:
compiled_routine.resource_values

{'active_volume': 196*M_r + 220*N_spatial + 240*b_as + 110*b_givens*(-1 + (b_givens*(N_spatial - 1)*(calc_lambda(M_r*R, b_givens*(N_spatial - 1)) - 1) + b_givens*(N_spatial - 1))/(b_givens*(N_spatial - 1)))*(N_spatial - 1) + 6*b_givens*(N_spatial - 1)*(calc_lambda(M_r*R, b_givens*(N_spatial - 1)) - 1) + 6*b_givens*(N_spatial - 1) + 994*b_mas + (-1 + (2*b_as + 2*ceiling(log2(R + 1)))/(b_as + ceiling(log2(R + 1))))*(55*b_as + 55*ceiling(log2(R + 1))) + 2*(-1 + (b_mas + (b_mas + ceiling(log2(R)))*(calc_lambda(M_r*R, b_mas + ceiling(log2(R))) - 1) + ceiling(log2(R)))/(b_mas + ceiling(log2(R))))*(55*b_mas + 55*ceiling(log2(R))) + (N_spatial - 1)*(108*Max(0, b_mas - Max(b_as, b_givens, b_mas)) + 108*Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas)) + 108*Max(b_as, b_givens, b_mas) - 48) + (N_spatial - 1)*(108*Max(0, b_mas - Max(b_as, b_givens, b_mas)) + 108*Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas)) + 10

In [2]:
compiled_routine.resource_values["rotations"]

(N_spatial - 1)*Max(0, Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas)) - Heaviside(-Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas) + 1.5, 0)*Heaviside(Max(0, b_mas - Max(b_as, b_givens, b_mas)) + Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas)) + Max(b_as, b_givens, b_mas) - 1.5, 0) - Heaviside(-Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas) + 2.5, 0)*Heaviside(Max(0, b_mas - Max(b_as, b_givens, b_mas)) + Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas)) + Max(b_as, b_givens, b_mas) - 2.5, 0)) + (N_spatial - 1)*Max(0, Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas)) - Max(0, b_mas - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b

In [2]:
from bartiq.analysis import sympy_rewriter

rewriter = sympy_rewriter(compiled_routine.resource_values["active_volume"])

NameError: name 'compiled_routine' is not defined

In [ ]:
rewriter.substitute("min(2, $x)", "2").substitute("ntz(R+1)", "2").substitute(
    "max(b_as, b_givens, b_mas)", "b_givens"
).substitute("ceiling($x)", "x").expand().substitute("mod(1, $x)", "1").assume("2.5 - b_mas < 0").assume(
    "log2(R) > 1.5"
).assume(
    "b_givens > 10"
).assume(
    "b_mas>10"
).assume(
    "log2(R + 1) > 3.5"
)

SympyExpressionRewriter(expression=1.5*M_r*N_spatial*R*b_givens - 1.5*M_r*R*b_givens + 1.5*M_r*R*b_mas + 1.5*M_r*R*log2(R) + 3.0*M_r*R + 98*M_r*R/calc_lambda(M_r*R, N_spatial*b_givens - b_givens) + 98*M_r*R/calc_lambda(M_r*R, b_mas + log2(R)) + 196*M_r*R/calc_lambda(M_r*R, 1) + 196*M_r + 110*N_spatial**2*b_givens**2*calc_lambda(M_r*R, N_spatial*b_givens - b_givens)/(N_spatial*b_givens - b_givens) - 220*N_spatial*b_givens**2*calc_lambda(M_r*R, N_spatial*b_givens - b_givens)/(N_spatial*b_givens - b_givens) - 1.5*N_spatial*b_givens*calc_lambda(M_r*R, N_spatial*b_givens - b_givens)*select_elbow_const(M_r*R/calc_lambda(M_r*R, N_spatial*b_givens - b_givens), 0) + 6*N_spatial*b_givens*calc_lambda(M_r*R, N_spatial*b_givens - b_givens) + 3122*N_spatial*b_givens - 2800*N_spatial*b_mas + 28*N_spatial + 0.75*R*b_as + 0.75*R*log2(R + 1) + 0.75*R*log2(4*2**b_mas*2**log2(M_r)*2**log2(M_r*R - M_r) - 2*2**log2(M_r)*2**log2(M_r*R - M_r) + 2*2**log2(M_r)*M_r*R - 2*2**log2(M_r)*M_r + 2*M_r + 1) + 26.0*R +

In [ ]:
rewriter = sympy_rewriter(compiled_routine.resource_values["rotations"])
rewriter.substitute("max(b_as,b_givens, b_mas)", "b_as").assume("b_as>13").assume("-b_as+b_mas < 0").assume(
    "-b_as+b_givens < 0"
)

SympyExpressionRewriter(expression=b_as + (N_spatial - 1)*Max(0, -b_as + b_givens - Max(0, -b_as + b_mas)) + (N_spatial - 1)*Max(0, -b_as + b_givens - Max(0, -b_as + b_mas) - Max(0, -b_as + b_givens - Max(0, -b_as + b_mas)) - Max(0, -b_as + b_mas - Max(0, -b_as + b_mas) - Max(0, -b_as + b_givens - Max(0, -b_as + b_mas)))) + Max(0, -b_as + b_mas) + Max(0, -b_as + b_mas - Max(0, -b_as + b_mas) - Max(0, -b_as + b_givens - Max(0, -b_as + b_mas))) + 1, _original_expression=(N_spatial - 1)*Max(0, -Heaviside(-Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas) + 1.5, 0)*Heaviside(Max(0, b_mas - Max(b_as, b_givens, b_mas)) + Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas)) + Max(b_as, b_givens, b_mas) - 1.5, 0) - Heaviside(-Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_givens, b_mas) + 2.5, 0)*Heaviside(Max(0, b_mas - Max(b_as, b_givens, b_mas)) + Max(0, b_givens - Max(0, b_mas - Max(b_as, b_givens, b_mas)) - Max(b_as, b_

In [32]:
assignments = {"b_as": 10, "b_mas": 10, "b_givens": 13}
rewriter.evaluate_expression(assignments=assignments)

14

In [3]:
rewriter = sympy_rewriter("max(0, a+b)")
rewriter.assume("a>0").assume("a+b>0")

SympyExpressionRewriter(expression=a + b, _original_expression=Max(0, a + b), backend=<bartiq.symbolics.sympy_backend.SympyBackend object at 0x124c19670>, linked_symbols={}, _previous=(Assumption(symbol_name='a+b', comparator='>', value=0), SympyExpressionRewriter(expression=Max(0, a + b), _original_expression=Max(0, a + b), backend=<bartiq.symbolics.sympy_backend.SympyBackend object at 0x124c19670>, linked_symbols={}, _previous=(Assumption(symbol_name='a', comparator='>', value=0), SympyExpressionRewriter(expression=Max(0, a + b), _original_expression=Max(0, a + b), backend=<bartiq.symbolics.sympy_backend.SympyBackend object at 0x124c19670>, linked_symbols={}, _previous=(Initial(), None))))))